In this notebook we'll try to create NER system.
The purpose of this system is to extract Persons, Organizations and Locations

![img](http://researchkb.files.wordpress.com/2014/02/ner.png)

In [1]:
import sys, json, codecs, csv
import numpy as np
import random
import pandas as pd
import tqdm
import itertools
import seaborn as sns

# visualization
import matplotlib
import numpy as np
from sklearn.model_selection import train_test_split
from itertools import chain
import matplotlib.pyplot as plt
%matplotlib inline  

Import of keras layers

In [2]:
from keras.layers import Input, LSTM, Embedding, Dense
from keras.layers.wrappers import Bidirectional
from keras.models import Model

Using TensorFlow backend.


dataset can be downloaded from this source:
https://github.com/EuropeanaNewspapers/ner-corpora

In [ ]:
with open('../data/ner-corpora/enp_FR.bnf.bio/enp_FR.bnf.bio') as f:
    text = f.read()

In [ ]:
print(text[:100])

In [ ]:
words = []
types = []
for item in text.split('\n'):
    item = item.strip()
    if len(item) == 0:
        continue
    [w, t] = item.split(' ')
    words.append(w)
    types.append(t)

In [ ]:
unique_types = list(set(types))
type2id = {x:index for index, x in enumerate(unique_types)}
print(type2id)

In [ ]:
from collections import Counter
word2count = Counter(words)
MAX_WORD_COUNT = 30000
top_words = [x[0] for x in sorted(word2count.items(), key=lambda x: x[1], reverse=True)][:MAX_WORD_COUNT]
word2id = {x:index+1 for index, x in enumerate(top_words)}

In [ ]:
train_dataset, test_dataset = train_test_split(zip(words, types), train_size=0.7)

In [ ]:
input = Input(shape=(None,))
out = Embedding(input_dim=len(word2id)+1, output_dim=200)(input)
out = Bidirectional(LSTM(200, activation='relu', return_sequences=True))(out)
out = Dense(len(type2id), activation='softmax')(out)
model = Model(input, out)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

In [ ]:
def getWordId(w):
    return 0 if not w in word2id else word2id[w]

def gen_batches(dataset, batch_size=64, seq_size=32, batch_count=100):
    random.shuffle(dataset)
    
    features = np.zeros((batch_size, seq_size))
    labels = np.zeros((batch_size, seq_size, len(type2id)))
    for _ in range(batch_count):
        for seq_index in range(batch_size):
            left = random.randint(0, len(dataset) - seq_size)
            features[seq_index,:] = [getWordId(x[0]) for x in dataset[left:left+seq_size]]
            labels[seq_index,:] = 0
            for i,(_,t) in enumerate(dataset[left:left+seq_size]):
                labels[seq_index,i] = 0
                labels[seq_index,i,type2id[t]] = 1
        yield features, labels
        
def encode_text(sentence):
    words = sentence.split()
    result = np.zeros((len(words),))
    for i,w in enumerate(words):
        result[i] = getWordId(w)
    return result

In [ ]:
from keras.callbacks import TensorBoard
import tensorflow as tf
import os, shutil


def write_log(callback, names, logs, batch_no):
    for name, value in zip(names, logs):
        summary = tf.Summary()
        summary_value = summary.value.add()
        summary_value.simple_value = value
        summary_value.tag = name
        callback.writer.add_summary(summary, batch_no)
        callback.writer.flush()
        
logs_dir = './logs'
callback = TensorBoard(logs_dir)
callback.set_model(model)

In [ ]:
unique_types

In [ ]:
for epoch in range(100):
    losses = []
    for x,y in gen_batches(train_dataset, batch_count=32):
        loss = model.train_on_batch(x, y)
        losses.append(loss)
    train_loss = np.mean(losses)
        
    losses = []
    for x,y in gen_batches(test_dataset, batch_count=32):
        loss = model.test_on_batch(x, y)
        losses.append(loss)
    test_loss = np.mean(losses)
    
    write_log(callback, ['train', 'test'], [train_loss, test_loss], epoch)

# Test model

Let's review how model works in production!

In [ ]:
query = test_dataset[160:260]
query_words = [x[0] for x in query]
query_types = [x[1] for x in query]
result = model.predict_on_batch(encode_text(" ".join(query_words)).reshape((1, -1)))[0]
for index in range(result.shape[0]):
    w = query_words[index]
    true_type = query_types[index]
    pred_type = unique_types[np.argmax(result[index,:])] 
    print("{}:\t{}\t{}".format(w, pred_type, true_type))

# Home task

- 3 points: make the model better
- 7 points: implement the model with CRF layer (https://github.com/Hironsan/keras-crf-layer)